In [21]:
!pip install --quiet "langchain < 0.99" "openai < 1.16.99" "httpx==0.27.2" "langchain_google_genai"

In [22]:

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain, SequentialChain

In [23]:
class TravelTemplate:
    def __init__(self):
        self.system_template = """
        You are a Brazilian travel agent that give good advices about travels around the world.
        Customer request will be denoted by 4 hashtags.

        Give your answer as list of places that they should visit. 

        For example:
        #### O que fazer no Rio de Janeiro?
           - no primeiro dia, checking no hotem
           - vai para a praia de copacabana e experimente agua de coco
        """

        self.human_template = """
        ####{request}
        """

        self.system_message_prompt = SystemMessagePromptTemplate.from_template(self.system_template)
        self.human_message_prompt = HumanMessagePromptTemplate.from_template(self.human_template)
        self.chat_prompt = ChatPromptTemplate.from_messages([self.system_message_prompt, 
                                                            self.human_message_prompt])
    



In [24]:
!pip install dotenv

In [25]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_GOOGLE")

travel_prompt = TravelTemplate()
chat_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.1, google_api_key=api_key)

In [26]:
parser = LLMChain(
    llm=chat_model,
    prompt=travel_prompt.chat_prompt,
    output_key="travel_tips"
)

chain = SequentialChain(
    chains=[parser],
    input_variables=["request"],
    output_variables=["travel_tips"],
    verbose=True
)


In [27]:
response = chain(
    {"request": "Gostaria de dicas de viagens para Paris em Julho. Viajarei 10 dias e gostaria de aproveitar a cidade"},
    return_only_outputs=True
)

print(response["travel_tips"])



> Entering new SequentialChain chain...

> Finished chain.
        Olá! Paris em julho é uma excelente escolha! Aqui estão algumas dicas para aproveitar ao máximo seus 10 dias na cidade luz:
        
        - **Torre Eiffel:** Comece sua viagem com o ícone de Paris. Reserve seus ingressos com antecedência para evitar filas e aproveite a vista panorâmica da cidade.
        - **Museu do Louvre:** Dedique pelo menos meio dia para explorar este museu gigante. Não perca a Mona Lisa, a Vênus de Milo e outras obras-primas.
        - **Catedral de Notre-Dame:** Mesmo em reconstrução, a catedral é um marco histórico impressionante. Caminhe pela área e admire sua arquitetura gótica.
        - **Champs-Élysées e Arco do Triunfo:** Passeie pela famosa avenida Champs-Élysées, repleta de lojas e restaurantes, e chegue ao Arco do Triunfo, de onde você pode ter uma vista incrível da cidade.
        - **Bairro de Montmartre e Sacré-Cœur:** Explore o charmoso bairro de Montmartre, lar de artistas e d

In [28]:
import logging
logging.basicConfig(level=logging.INFO)

In [29]:
class Agent:
    def __init__(self, key, model="gemini-2.0-flash", temperature=0.1):
        self.key = key
        self.model = model
        self.temperature = temperature
        self.logger = logging.getLogger(__name__)
        self.chat_model = ChatGoogleGenerativeAI(model=model, temperature=temperature, google_api_key=key)

    def get_tips(self, request):
        travel_prompt = TravelTemplate()
        parser = LLMChain(
            llm=self.chat_model,
            prompt=travel_prompt.chat_prompt,
            output_key="travel_tips"
        )

        chain = SequentialChain(
            chains=[parser],
            input_variables=["request"],
            output_variables=["travel_tips"],
            verbose=True
        )
        return chain({"request": request}, return_only_outputs=True)

In [30]:
pedido = "Gostaria de viajar para Itu em São Paulo durante 10 dias. Gostaria de aproveitar a cidade e a região"

agent = Agent(api_key)

response = agent.get_tips(pedido)
print(response['travel_tips'])



> Entering new SequentialChain chain...

> Finished chain.
        Olá! Itu é uma cidade encantadora com muita história e beleza natural. Dez dias é um ótimo tempo para explorar a região com calma. Aqui está uma sugestão de roteiro para você aproveitar ao máximo:
        
        1.  **Centro Histórico de Itu:**
            *   Comece explorando o centro histórico, onde você encontrará o famoso "Orelhão Gigante" e outros monumentos históricos.
            *   Visite a Igreja Matriz de Nossa Senhora da Candelária, um belo exemplo da arquitetura religiosa da região.
            *   Passeie pela Praça da Independência e admire os casarões antigos.
        
        2.  **Museu da Energia:**
            *   Conheça a história da energia no Brasil e a importância de Itu nesse contexto.
            *   O museu oferece exposições interativas e educativas para todas as idades.
        
        3.  **Fazenda do Chocolate:**
            *   Desfrute de um dia em contato com a natureza, saborean